In [22]:
import numpy as np
from MLP.model import MLP
import pandas as pd

# 导入npy数据集
X_train = np.load('Assignment1-Dataset/train_data.npy')
X_test = np.load('Assignment1-Dataset/test_data.npy')
Y_train = np.load('Assignment1-Dataset/train_label.npy')
Y_test = np.load('Assignment1-Dataset/test_label.npy')


In [23]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((50000, 128), (50000, 1), (10000, 128), (10000, 1))

In [24]:
Y_train = pd.Series(Y_train.ravel())  # 或者使用 Y_train.squeeze()
Y_train

0        6
1        9
2        9
3        4
4        1
        ..
49995    2
49996    6
49997    9
49998    1
49999    1
Length: 50000, dtype: uint8

In [25]:


# 超参数
input_dim = 128  # 输入数据维度
hidden_dims = [64, 32]  # 隐藏层维度
output_dim = 10  # 输出类别数（例如10分类）
learning_rate = 0.01
momentum = 0.9
num_epochs = 20
batch_size = 32

# 创建MLP模型
model = MLP(input_dim, hidden_dims, output_dim, dropout_rate=0.5, weight_decay=1e-4)

# 初始化动量优化需要的速度
velocity = {}

# 训练循环
for epoch in range(num_epochs):
    
    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        Y_batch = Y_train[i:i+batch_size]
        
        # 前向传播
        logits = model.forward(X_batch, training=True)
        
        # 计算损失
        loss = model.compute_loss(logits, Y_batch)
        epoch_loss += loss
        
        # 反向传播
        model.backward()
        
        # 更新参数
        model.update(learning_rate, momentum, velocity)
    
    # 打印每个epoch的损失
    avg_loss = epoch_loss / (len(X_train) // batch_size)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')



ValueError: operands could not be broadcast together with shapes (32,) (32,10) 

In [20]:
Y_test = pd.Series(Y_train.ravel())  # 或者使用 Y_train.squeeze()
Y_test

C:\Users\luo\AppData\Local\Temp\ipykernel_38392\4058059862.py:1: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  Y_test = pd.Series(Y_train.ravel())  # 或者使用 Y_train.squeeze()


0        6
1        9
2        9
3        4
4        1
        ..
49995    2
49996    6
49997    9
49998    1
49999    1
Length: 50000, dtype: uint8

In [18]:
# 测试阶段（演示）
logits_test = model.forward(X_test, training=False)
predictions = np.argmax(logits_test, axis=1)
print("Predictions:", predictions)


Predictions: [2 3 3 8 5 3 3 3 3 3 3 3 3 3 3 2 3 5 1 3 5 3 4 9 9 4 5 3 4 6 3 3 6 3 2 6 3
 2 6 3 3 3 3 4 4 5 4 3 4 3 3 3 1 3 3 7 3 9 3 9 7 3 8 2 2 3 4 3 8 3 9 3 3 3
 3 6 3 2 3 3 3 3 3 3 4 3 3 3 3 3 3 3 6 6 5 3 4 3 3 3]


In [21]:
# 计算准确率
correct_predictions = np.sum(predictions == Y_test)
accuracy = correct_predictions / len(Y_test)
print(f'Accuracy: {accuracy:.4f}')

ValueError: ('Lengths must match to compare', (50000,), (100,))